### CSV vs PARQUET (Arrow) ###

- escalabilidade do formato

- usando DuckDB local

- com MovieLens 100k e 33M

In [1]:
# ===============================================
#   BLOCO INICIAL — IMPORTS & PATHS & SETUP
# ===============================================

import duckdb
import pandas as pd
import polars as pl
import time
from pathlib import Path

# === PATHS PARA OS DATASETS ===

## MovieLens 100k
DATA_100k = Path("..") / "data" / "100K"
ratings_100k = DATA_100k / "ratings.csv"
movies_100k  = DATA_100k / "movies.csv"
tags_100k    = DATA_100k / "tags.csv"
links_100k   = DATA_100k / "links.csv"

## MovieLens 33M
DATA_33m = Path("..") / "data" / "Full33M"
ratings_33m = DATA_33m / "ratings.csv"
movies_33m  = DATA_33m / "movies.csv"
tags_33m    = DATA_33m / "tags.csv"
links_33m   = DATA_33m / "links.csv"

print("Paths definidos com sucesso.")

# === Conexão DuckDB local (ficheiro persistente opcional) ===
con = duckdb.connect("movielens_local.duckdb")
print("Conexão DuckDB aberta.")

Paths definidos com sucesso.
Conexão DuckDB aberta.


In [2]:
print(ratings_100k)
print(ratings_33m)

..\data\100K\ratings.csv
..\data\Full33M\ratings.csv


In [3]:
# Convert from csv to parquet

duckdb.sql(f"""
COPY (SELECT * FROM '{ratings_100k}')
TO '{ratings_100k.with_suffix(".parquet")}'
(FORMAT 'parquet');
""")

duckdb.sql(f"""
COPY (SELECT * FROM '{ratings_33m}')
TO '{ratings_33m.with_suffix(".parquet")}'
(FORMAT 'parquet');
""")

print("Conversão concluída.")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Conversão concluída.


In [4]:
def medir_tempo(query):
    t0 = time.time()
    duckdb.sql(query).df()
    return round(time.time() - t0, 3)


In [5]:
# medicao real

tempos = {
    "CSV_100k": medir_tempo(f"""
        SELECT movieId, AVG(rating)
        FROM '{ratings_100k}'
        GROUP BY movieId
    """),

    "PARQUET_100k": medir_tempo(f"""
        SELECT movieId, AVG(rating)
        FROM '{ratings_100k.with_suffix('.parquet')}'
        GROUP BY movieId
    """),

    "CSV_33M": medir_tempo(f"""
        SELECT movieId, AVG(rating)
        FROM '{ratings_33m}'
        GROUP BY movieId
    """),

    "PARQUET_33M": medir_tempo(f"""
        SELECT movieId, AVG(rating)
        FROM '{ratings_33m.with_suffix('.parquet')}'
        GROUP BY movieId
    """),
}

tempos


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

{'CSV_100k': 0.073,
 'PARQUET_100k': 0.014,
 'CSV_33M': 2.092,
 'PARQUET_33M': 0.242}

In [6]:
# Tabela comparativa

df_tempo = pd.DataFrame([
    ["100k", tempos["CSV_100k"], tempos["PARQUET_100k"]],
    ["33M", tempos["CSV_33M"], tempos["PARQUET_33M"]],
], columns=["Dataset", "Tempo CSV (s)", "Tempo Parquet (s)"])

df_tempo


,Dataset,Tempo CSV (s),Tempo Parquet (s)
0,100k,0.073,0.014
1,33M,2.092,0.242


__Close the connection (when done)__

In [7]:
#con.close()
#print("Connection closed.")

### DuckDB vs Polars ###

- mesmo cálculo

- em DuckDB e Polars

- usando Parquet apenas (formato colunar ideal)

In [8]:
# paths para Parquet

p100k = ratings_100k.with_suffix(".parquet")
p33m = ratings_33m.with_suffix(".parquet")

In [12]:
# Funções de benchmark

def run_duckdb(path):
    t0 = time.time()
    duckdb.sql(f"""
        SELECT movieId, AVG(rating)
        FROM '{path}'
        GROUP BY movieId
    """).df()
    return round(time.time() - t0, 3)

def run_polars(path):
    t0 = time.time()
    (
        pl.scan_parquet(str(path))     
          .group_by("movieId")         
          .agg(pl.col("rating").mean())
          .collect()
    )
    return round(time.time() - t0, 3)


In [13]:
# Comparação

df_duck_polars = pd.DataFrame([
    ["100k", run_duckdb(p100k), run_polars(p100k)],
    ["33m", run_duckdb(p33m), run_polars(p33m)],
], columns=["Dataset", "DuckDB (s)", "Polars (s)"])

df_duck_polars


,Dataset,DuckDB (s),Polars (s)
0,100k,0.005,0.011
1,33m,0.150,0.462


### Athena + Polars

- query distribuída via Athena

- análise local via Polars

In [1]:
#Definir query (mesma lógica usada em DuckDB / Polars locais)

QUERY_ATHENA = f"""
SELECT 
    movieId,
    AVG(rating) AS avg_rating,
    COUNT(*)    AS num_ratings
FROM {DB_ATHENA}.ratings_32m
GROUP BY movieId
"""

NameError: name 'DB_ATHENA' is not defined

In [ ]:
# Executar query no Athena + medir tempo

start = time.time()

df_athena_pd = read_sql_df(QUERY_ATHENA, conn)

athena_time = round(time.time() - start, 2)
print(f"Tempo Athena: {athena_time}s")


In [ ]:
df_athena_pd.head()


In [ ]:
#Converter para Polars para pós-processamento

df_athena_pl = pl.from_pandas(df_athena_pd)
df_athena_pl.head()


In [ ]:
top_movies_athena_polars = (
    df_athena_pl
    .filter(pl.col("num_ratings") > 100)
    .sort("avg_rating", descending=True)
    .head(10)
)

top_movies_athena_polars


In [ ]:
#Integração com a tabela final de comparação

df_comparacao = pd.DataFrame([
    ["100k local",  "DuckDB",  duckdb_100k_time],
    ["33M local",   "DuckDB",  duckdb_33m_time],
    ["33M local",   "Polars",  polars_33m_time],
    ["32M cloud",   "Athena",  athena_time],
], columns=["Dataset", "Engine", "Tempo (s)"])

df_comparacao
